In [1]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [40]:
import pandas as pd
import numpy as np
import pickle
import yfinance as yf
import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [41]:
# Mengambil data harga minyak mentah WTI dari Yahoo Finance
wti_data = yf.download('CL=F', start='2000-01-01', end='2023-12-29')
wti_data = wti_data[['Close']]
wti_data = wti_data.dropna()

[*********************100%%**********************]  1 of 1 completed


In [42]:
wti_data.tail()

,Close
Date,
2023-12-21,73.889999
2023-12-22,73.559998
2023-12-26,75.570000
2023-12-27,74.110001
2023-12-28,71.769997


In [43]:
#Memuat model
with open('model/crude-oil-lstm-fix2.pkl', 'rb') as f:
    lstm_model = pickle.load(f)
#Memuat model
with open('model/crude-oil-gru-fix2.pkl', 'rb') as f:
    gru_model = pickle.load(f)
#Memuat model
with open('model/crude-oil-prophet-fix.pkl', 'rb') as f:
    prophet_model = pickle.load(f)

D:\Mentahan\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 16 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [44]:
# Menyimpan data untuk model LSTM dan GRU

#Mengubah data menjadi arrya numpy dan menskalanya
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(wti_data)

#fungsi untuk membuat dataset dengan sliding window
def create_dataset(data, time_step):
    X, Y = [],[]
    for i in range(len(data)-time_step-1):
        a = data[i:(i+time_step),0]
        X.append(a)
        Y.append(data[i + time_step,0])
    return np.array(X), np.array(Y)

#menyiapkan data untuk LSTM dan GRU
time_step = 60 #misalnya menggunakan 60 hari terakhir untuk memprediksi
X, y = create_dataset(scaled_data, time_step)

#mengubha bentuk data menjadi [samples, time steps, features]
X = X.reshape(X.shape[0], X.shape[1],1)

In [45]:
# Melakukan prediksi
def predict_future(model, data, n_days, time_step):
    predictions = []
    last_data = data[-time_step:]

    for i in range(n_days):
        last_data = last_data.reshape((1, time_step,1))
        pred = model.predict(last_data)
        predictions.append(pred[0,0])

        pred = np.array([pred[0, 0]]).reshape(1, 1, 1)
        last_data = np.append(last_data[:, 1:, :], pred, axis=1)

    return predictions

#prediksi untuk 7 dan 14 hari ke depan
lstm_predictions_7 = predict_future(lstm_model, scaled_data, 7, time_step)
lstm_predictions_14 = predict_future(lstm_model, scaled_data, 14, time_step)

gru_predictions_7 = predict_future(gru_model, scaled_data, 7, time_step)
gru_predictions_14 = predict_future(gru_model, scaled_data, 14, time_step)

#menginversi skala prediksi ke bentuk aslinya
lstm_predictions_7 = scaler.inverse_transform(np.array(lstm_predictions_7).reshape(-1,1))
lstm_predictions_14 = scaler.inverse_transform(np.array(lstm_predictions_14).reshape(-1,1))

gru_predictions_7 = scaler.inverse_transform(np.array(gru_predictions_7).reshape(-1,1))
gru_predictions_14 = scaler.inverse_transform(np.array(gru_predictions_14).reshape(-1,1))


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━

In [46]:
lstm_predictions_7

array([[82.0665  ],
       [81.42896 ],
       [80.77598 ],
       [80.15267 ],
       [79.60764 ],
       [79.158356],
       [78.804375]], dtype=float32)

In [47]:
dates_7 = pd.date_range(start=wti_data.index[-1] + pd.Timedelta(days=1), periods=7, freq='D')
dates_14 = pd.date_range(start=wti_data.index[-1] + pd.Timedelta(days=1), periods=14, freq='D')

# Membuat DataFrame untuk prediksi
lstm_predictions_df_7 = pd.DataFrame(lstm_predictions_7, columns=['Predicted'], index=dates_7)
lstm_predictions_df_14 = pd.DataFrame(lstm_predictions_14, columns=['Predicted'], index=dates_14)

gru_predictions_df_7 = pd.DataFrame(gru_predictions_7, columns=['Predicted'], index=dates_7)
gru_predictions_df_14 = pd.DataFrame(gru_predictions_14, columns=['Predicted'], index=dates_14)

In [48]:
lstm_predictions_df_7

,Predicted
2023-12-29,82.066498
2023-12-30,81.428963
2023-12-31,80.775978
2024-01-01,80.152672
2024-01-02,79.607643
2024-01-03,79.158356
2024-01-04,78.804375


In [49]:
lstm_predictions_df_14

,Predicted
2023-12-29,82.066498
2023-12-30,81.428963
2023-12-31,80.775978
2024-01-01,80.152672
2024-01-02,79.607643
2024-01-03,79.158356
2024-01-04,78.804375
2024-01-05,78.536118
2024-01-06,78.340225
2024-01-07,78.202591


In [50]:
gru_predictions_df_7

,Predicted
2023-12-29,598.442078
2023-12-30,604.420166
2023-12-31,608.546570
2024-01-01,612.438416
2024-01-02,616.240417
2024-01-03,619.983154
2024-01-04,623.674927


In [51]:
gru_predictions_df_14

,Predicted
2023-12-29,598.442078
2023-12-30,604.420166
2023-12-31,608.546570
2024-01-01,612.438416
2024-01-02,616.240417
2024-01-03,619.983154
2024-01-04,623.674927
2024-01-05,627.317932
2024-01-06,630.915649
2024-01-07,634.469177


In [60]:
# Melakukan prediksi dengan prophet mode;
import prophet
from prophet import Prophet

wti_data.reset_index(inplace=True)
wti_data.rename(columns={'Date' : 'ds', 'Close' : 'y'}, inplace=True)

# Pastikan data asli memiliki frekuensi harian
wti_data.set_index('ds', inplace=True)
wti_data = wti_data.asfreq('D')

# Melakukan prediksi dengan Prophet
#prophet_model.fit(wti_data)

# Tentukan tanggal awal untuk prediksi
#last_date = wti_data.index[-1]
last_date = wti_data.index[-1]
future_7 = pd.date_range(start=last_date + pd.DateOffset(days=1), periods=24, freq='M')
future_14 = pd.date_range(start=last_date + pd.DateOffset(days=1), periods=48, freq='M')
future_7 = prophet_model.make_future_dataframe(periods=24, freq='M')#, include_history=False, start_date=last_date + pd.DateOffset(days=1))
future_14 = prophet_model.make_future_dataframe(periods=48, freq='M')#, include_history=False, start_date=last_date + pd.DateOffset(days=1))

forecast_7 = prophet_model.predict(future_7)
forecast_14 = prophet_model.predict(future_14)

# Mengambil nilai prediksi
prophet_predictions_7 = forecast_7[['ds','yhat']].tail()
prophet_predictions_14 = forecast_14[['ds','yhat']].tail()

In [61]:
prophet_predictions_7

,ds,yhat
5170,2021-08-31,75.459519
5171,2021-09-30,74.953226
5172,2021-10-31,72.622345
5173,2021-11-30,72.521451
5174,2021-12-31,73.344871


In [62]:
prophet_predictions_14

,ds,yhat
5194,2023-08-31,86.567884
5195,2023-09-30,81.871999
5196,2023-10-31,84.271190
5197,2023-11-30,83.707025
5198,2023-12-31,82.941685
